In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from pathlib import Path
import os

In [6]:
# Check and change the current working directory - A.F.
print(os.getcwd())
os.chdir(Path().resolve())

C:\Users\turks\OneDrive\Documents\Github 2\DSE6311OM_Group3\Notebook


In [7]:
# read in data set with first column (district name) set to index
df = pd.read_excel("../Data/cleaned_data_encoded.xlsx", index_col =0)
df.head()

,english_learners_pct,students_with_disabilities_pct,progress_toward_improvement_targets_pct,in_district_expenditures,in_district_expenditures_per_pupil,graduation_rate_pct,still_in_school_pct,non_grad_completers_pct,hs_equivalency_pct,dropout_rate_pct,...,overall_classification_Not requiring assistance or intervention,overall_classification_Requiring assistance or intervention,reason_for_classification_In need of broad/comprehensive support,reason_for_classification_In need of focused/targeted support,reason_for_classification_Limited or no progress toward targets,reason_for_classification_Meeting or exceeding targets,reason_for_classification_Moderate progress toward targets,reason_for_classification_Substantial progress toward targets,log_in_district_expenditures,needs_income_avg_pct
district_name,,,,,,,,,,,,,,,,,,,,,
Abington,11.0,17.8,39,3.560342e+07,16609.95,88.9,1.3,0.7,0.0,9.2,...,1,0,0,0,0,0,1,0,17.387952,40.85
Acton-Boxborough,6.2,15.5,85,1.005648e+08,19407.68,96.1,3.2,0.2,0.0,0.4,...,1,0,0,0,0,1,0,0,18.426312,20.10
Agawam,6.2,18.3,46,7.195848e+07,20414.91,88.0,1.1,1.9,0.4,8.6,...,1,0,0,0,0,0,1,0,18.091600,46.05
Amesbury,2.7,23.7,47,3.996617e+07,22338.70,88.3,2.5,0.8,0.8,7.5,...,1,0,0,0,0,0,1,0,17.503544,40.10
Amherst-Pelham,7.3,24.2,33,3.248902e+07,26177.60,90.9,4.1,0.9,0.9,3.2,...,0,1,0,1,0,0,0,0,17.296413,36.90


I would like to split each of these columns into three sections - Low, Moderate, and High.
To do so, I will use tertiles.  From there I would like to define a new variable called 'needs_income' that will have the following values to describe the school districts overall level of student needs and municipality average income:
"Low Needs, Low Income"
"Low Needs, Moderate Income"
"Low Needs, High Income"
"Moderate Needs, Low Income"
"Moderate Needs, Moderate Income"
"Moderate Needs, High Income"
"High Needs, Low Income"
"High Needs, Moderate Income"
"High Needs, High Income"

percentiles = {
    'high_needs_pct': df['high_needs_pct'].quantile([1/3, 2/3]),
    'low_income_pct': df['low_income_pct'].quantile([1/3, 2/3])
}

print("\nPercentiles (1/3 and 2/3):")
print(percentiles)


# Clean column names and ensure numeric types
df.columns = df.columns.str.strip()
df['high_needs_pct'] = pd.to_numeric(df['high_needs_pct'], errors='coerce')
df['low_income_pct'] = pd.to_numeric(df['low_income_pct'], errors='coerce')

conditions = [
    (df['high_needs_pct'] <= 35.2) & (df['low_income_pct'] > 39.4),
    (df['high_needs_pct'] <= 35.2) & (df['low_income_pct'] > 21.6) & (df['low_income_pct'] <= 39.4),
    (df['high_needs_pct'] <= 35.2) & (df['low_income_pct'] <= 21.6),
    (df['high_needs_pct'] > 35.2) & (df['high_needs_pct'] <= 50.9) & (df['low_income_pct'] > 39.4),
    (df['high_needs_pct'] > 35.2) & (df['high_needs_pct'] <= 50.9) & (df['low_income_pct'] > 21.6) & (df['low_income_pct'] <= 39.4),
    (df['high_needs_pct'] > 35.2) & (df['high_needs_pct'] <= 50.9) & (df['low_income_pct'] <= 21.6),
    (df['high_needs_pct'] > 50.9) & (df['low_income_pct'] > 39.4),
    (df['high_needs_pct'] > 50.9) & (df['low_income_pct'] > 21.6) & (df['low_income_pct'] <= 39.4),
    (df['high_needs_pct'] > 50.9) & (df['low_income_pct'] <= 21.6),
]

choices = [
    "Low Needs, Low Income",
    "Low Needs, Moderate Income",
    "Low Needs, High Income",
    "Moderate Needs, Low Income",
    "Moderate Needs, Moderate Income",
    "Moderate Needs, High Income",
    "High Needs, Low Income",
    "High Needs, Moderate Income",
    "High Needs, High Income"
]

df['needs_income'] = np.select(conditions, choices, default="Unclassified")

# Check results
df.head()

print(df[['high_needs_pct', 'low_income_pct', 'needs_income']].head())